# JanusGraph Quick Start

**File**: notebooks/01_quickstart.ipynb  
**Created**: 2026-01-28T11:10:00.123  
**Author**: David LECONTE, IBM WorldWide | Data & AI

---

## Quick Introduction

This notebook demonstrates:
1. Connecting to JanusGraph
2. Basic queries
3. Simple visualization

**Prerequisites**: Stack must be running (`make deploy`)

In [1]:
# Fix event loop for Jupyter
import nest_asyncio
nest_asyncio.apply()

# Core imports
from gremlin_python.driver import client
import pandas as pd

print("✅ Libraries imported")

✅ Libraries imported


In [2]:
# Connect to JanusGraph
import os
GREMLIN_URL = os.getenv('GREMLIN_URL', 'ws://localhost:18182/gremlin')
gc = client.Client(GREMLIN_URL, 'g')

# Test connection
v_count = gc.submit('g.V().count()').all().result()[0]
e_count = gc.submit('g.E().count()').all().result()[0]

print(f"✅ Connected to JanusGraph")
print(f"   Vertices: {v_count}")
print(f"   Edges: {e_count}")

✅ Connected to JanusGraph
   Vertices: 11
   Edges: 19


## Query Examples

In [3]:
# Get all people
query = "g.V().hasLabel('person').valueMap('full_name', 'age', 'nationality')"
result = gc.submit(query).all().result()

people = []
for p in result:
    people.append({
        'Name': p.get('full_name', [''])[0],
        'Age': p.get('age', [''])[0],
        'Nationality': p.get('nationality', [''])[0]
    })

df = pd.DataFrame(people)
print("People in the graph:")
display(df)

People in the graph:


,Name,Age,Location
0,Eve Davis,32,Austin
1,Carol Williams,35,Seattle
2,Bob Smith,25,New York
3,Alice Johnson,30,San Francisco
4,David Brown,28,San Francisco


In [4]:
# Get relationships
query = "g.E().groupCount().by(label)"
edges = gc.submit(query).all().result()[0]

df_edges = pd.DataFrame(list(edges.items()), columns=['Edge Type', 'Count'])
print("Relationships:")
display(df_edges)

Relationships:


,Edge Type,Count
0,created,3
1,worksFor,5
2,uses,5
3,knows,6


## Bonus: Real-Time Streaming with Pulsar

The project supports **event streaming** via Apache Pulsar for real-time data ingestion.

In [ ]:
# Quick streaming example
try:
    from banking.streaming import create_person_event, EntityProducer, MockEntityProducer
    
    # Create an event (same ID used in Pulsar, JanusGraph, and OpenSearch)
    event = create_person_event(
        person_id="quickstart-person-001",
        name="Quickstart Demo",
        payload={"id": "quickstart-person-001", "first_name": "Quick", "last_name": "Start"},
        source="QuickstartNotebook"
    )
    
    print("✅ Streaming available!")
    print(f"   Event ID: {event.entity_id}")
    print(f"   Topic: {event.get_topic()}")
    print("\n   See banking/notebooks/11_Streaming_Pipeline_Demo.ipynb for full demo")
except ImportError as e:
    print(f"⚠️ Streaming module not available: {e}")

## Next Steps

- **Graph Queries**: See `02_janusgraph_complete_guide.ipynb` for advanced queries
- **Streaming**: See `banking/notebooks/11_Streaming_Pipeline_Demo.ipynb` for Pulsar integration
- **Banking Demos**: See `banking/notebooks/` for compliance demos (AML, Fraud, etc.)

---

**Signature**: David LECONTE, IBM WorldWide | Data & AI

In [ ]:
3